In [22]:
class SearchEngineBase(object):
    def __init__(self):
        pass
    
    def add_corpus(self, file_path):
        with open(file_path, 'r') as fin:
            text = fin.read()
        self.process_corpus(file_path, text)
    
    def process_corpus(self, id, text):
        raise Exception('process_corpus not implemented.')
    
    def search(self, query):
        raise Exception('search not implemented.')

def main(search_engine):
    for file_path in ['1.txt', '2.txt', '3.txt', '4.txt', '5.txt']:
        search_engine.add_corpus(file_path)
    
    while True:
        query = input()
        results = search_engine.search(query)
        print('found {} result(s)'.format(len(results)))
        for result in results:
            print(result)

class SimpleEngine(SearchEngineBase):
    def __init__(self):
        super(SimpleEngine, self).__init__()
        self.__id_to_texts = {}
    
    def process_corpus(self, id, text):
        self.__id_to_texts[id] = text
    
    def search(self, query): 
        results = [] 
        for id, text in self.__id_to_texts.items(): 
            if query in text: 
                results.append(id) 
        return results

# search_engine = SimpleEngine()
# main(search_engine)

In [ ]:
import re

class BOWEngine(SearchEngineBase):
    def __init__(self):
        super().__init__()
        self.__id_to_words = {}
    
    def process_corpus(self, id, text):
        self.__id_to_words[id] = self.parse_text_to_words(text)
    
    def search(self, query):
        query_words = self.parse_text_to_words(query)
        results = []
        for id, words in self.__id_to_words.items():
            if self.query_match(query_words, words):
                results.append(id)
        return results
    
    @staticmethod
    def query_match(query_words, words):
        for query_word in query_words:
            if query_word not in words:
                return False
        return True
    
    @staticmethod
    def parse_text_to_words(text):
        # 使用正则表达式去除标点符号和换行符
        text = re.sub(r'[^\w]', ' ', text)
        # 转化小写
        text = text.lower()
        # 生成所有单词的列表
        word_list = text.split(' ')
        # 去除空白单词
        word_list = filter(None, word_list)
        # 返回单词list
        return set(word_list)
search_engine = BOWEngine()
main(search_engine)

 i have a dream


found 3 result(s)
1.txt
2.txt
3.txt


 I have a dream


found 3 result(s)
1.txt
2.txt
3.txt


 I HAVE A DREAM


found 3 result(s)
1.txt
2.txt
3.txt


 I HAVES A


found 0 result(s)


In [23]:
import re
s = "I'm a good student.and_888--9"
s = re.sub(r'[^\w]', ' ', s)
print(s.split(' '))
print(set(filter(None, s.split(' '))))

index = {
    "a": 1,
    "b": 2,
    "c": 3
}

# print(('a', 1) in index.keys)

print(list(enumerate('abc', start=-1)))


results = set() 
for idx, query_word in enumerate(query_words): # 获取每一个单词的存在逻辑 
    if idx == 0: 
        results = set(self.inverted_index[query_word]) 
    results = results.intersection(self.inverted_index[query_word]) 
return results


query_words = list(['I', 'am', 'a', 'good'])
query_words_index = list()
for query_word in query_words:
    query_words_index.append(0)
print(query_words_index)

['I', 'm', 'a', 'good', 'student', 'and_888', '', '9']
{'student', '9', 'm', 'a', 'I', 'and_888', 'good'}
[(-1, 'a'), (0, 'b'), (1, 'c')]
[0, 0, 0, 0]


In [24]:
import re

class BOWInvertedIndexEngine(SearchEngineBase):
    def __init__(self):
        super().__init__()
        self.__inverted_index = {}
    
    def process_corpus(self, id, text):
        words = self.parse_text_to_words(text)
        for word in words:
            if word not in self.__inverted_index:
                self.__inverted_index[word] = []
            self.__inverted_index[word].append(id)
            
    def search(self, query):
        query_words = list(self.parse_text_to_words(query))
        results = set() 
        for idx, query_word in enumerate(query_words): # 获取每一个单词的存在逻辑 
            if idx == 0: 
                results = set(self.__inverted_index[query_word]) 
            results = results.intersection(self.__inverted_index[query_word]) 
        return results
                
    def search2(self, query):
        query_words = list(self.parse_text_to_words(query))
        query_words_index = list()
        for query_word in query_words:
            query_words_index.append(0)
        # 如果某一个单词的倒序索引为空，立刻返回
        for query_word in query_words:
            if query_word not in self.__inverted_index:
                return []
        
        result = []
        while True:
            # 首先获得当前状态下所有倒序索引的index
            current_ids = []
            
            for idx, query_word in enumerate(query_words):
                current_index = query_words_index[idx]
                current_inverted_list = self.__inverted_index[query_word]
                
                # 已经遍历到某一个倒序索引的末尾，结束search
                if current_index >= len(current_inverted_list):
                    return result
                current_ids.append(current_inverted_list[current_index])
            
            # 然后，如果current_ids的所有元素都一样，那么表明这个单词在这个元素对应的文档中都出现了
            if all(x == current_ids[0] for x in current_ids):
                result.append(current_ids[0])
                query_words_index = [x + 1 for x in query_words_index]
                continue
            # 如果不是，我们就把最小元素加一
            min_val = min(current_ids)
            min_val_pos = current_ids.index(min_val)
            query_words_index[min_val_pos] += 1
    
    @staticmethod
    def parse_text_to_words(text):
        # 使用正则去除标点符号和换行符
        text = re.sub(r'[^\w]', ' ', text)
        # 转化为小写
        text = text.lower()
        # 生成所有单词列表
        word_list = text.split(' ')
        # 去除空白词
        word_list = filter(None, word_list)
        # 返回单词的set
        return set(word_list)

search_engine = BOWInvertedIndexEngine()
main(search_engine)

 little a


found 2 result(s)
1.txt
2.txt


 little aaaaaaaaaa


<class 'KeyError'>: 'aaaaaaaaaa'

In [ ]:
import pylru

class LRUCache(object):
    def __init__(self, size=32):
        self.cache = pylru.lrucache(size)
    
    def has(self, key):
        return key in self.cache

    def get(self, key):
        return self.cache[key]
    
    def set(self, key, value):
        self.cache[key] = value

class BOWInvertedIndexEngineWithCache(BOWInvertedIndexEngine, LRUCache):
    def __init__(self):
        BOWInvertedIndexEngine.__init__(self)
        LRUCache.__init__(self)
    
    def search(self, query):
        if self.has(query):
            print('cache hit')
            return self.get(query)
        result = super().search(query)
        self.set(query, result)
        return result

search_engine = BOWInvertedIndexEngineWithCache()
main(search_engine)
            

 little a


found 2 result(s)
1.txt
2.txt


 little a


cache hit
found 2 result(s)
1.txt
2.txt


In [13]:
# 测试

class A():
    def __init__(self):
        print('enter A')
        print('leave A')

class B(A):
    def __init__(self):
        print('enter B')
        super().__init__()
        print('leave B')

class C(A):
    def __init__(self):
        print('enter C')
        # super().__init__()
        print('leave C')

class D(B, C):
    def __init__(self):
        print('enter D')
        super().__init__()
        C.__init__(self)
        print('leave D') 

D()


enter D
enter B
enter C
leave C
leave B
enter C
leave C
leave D


In [14]:
# 调用父类属性方法
class Father():
    def __init__(self):
        self.a = 'aaa'
    def action(self):
        print('调用父类的方法')
        
class Son(Father):
    pass

son = Son()
son.action()
son.a

调用父类的方法


'aaa'

In [15]:
# 重写父类的属性
class Son(Father):
    def __init__(self):
        self.a = 'bbb'
    def action(self):
        print('子类重写父类的方法')
son = Son()
son.action()
son.a

子类重写父类的方法


'bbb'

In [18]:
class Father():
    def __init__(self):
        self.a = 'a'
class Son(Father):
    def __init__(self):
        super().__init__()

son = Son()
print(son.a)

a


In [19]:
class Base:
    def __init__(self):
        print('Base.__init__')
class A(Base):
    def __init__(self):
        Base.__init__(self)
        print('A.__init__')
class B(Base):
    def __init__(self):
        Base.__init__(self)
        print('B.__init__')
class C(A, B):
    def __init__(self):
        A.__init__(self)
        B.__init__(self)
        print('C.__init__')
c = C()

# 从左到右
# 深度优先

Base.__init__
A.__init__
Base.__init__
B.__init__
C.__init__


In [20]:
class Base:
    def __init__(self):
        print('Base.__init__')
class A(Base):
    def __init__(self):
        super().__init__()
        print('A.__init__')
class B(Base):
    def __init__(self):
        super().__init__()
        print('B.__init__')
class C(A, B):
    def __init__(self):
        super().__init__()
        print('C.__init__')
c = C()

Base.__init__
B.__init__
A.__init__
C.__init__
